In [1]:
import pandas as pd
import re
from imblearn.over_sampling import ADASYN, RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler

c:\Users\vi_ma\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
# Import data
train = pd.read_csv("../data/train_complete.csv")
test = pd.read_csv("../data/test_complete.csv")

## Resampling

In [117]:
# Dataprep - toda transformação deve ser criada sobre o teste e somente aplicada sobre o treino
def apply_random_oversampling(df, oversample_col='', exclude_category=None):
    '''
    df: dataframe with all columns
    oversample_col: column to apply the oversampling
    exclude_category: list of categories on oversample_col to set apart while oversampling
    '''
    df_cat = pd.DataFrame(columns = df.columns)
    df_ = df.copy()
    
    # Separando categorias específica se necessário
    if exclude_category is not None:
        
        for cat in exclude_category:
            
            df_cat = pd.concat([df_cat, df[df[oversample_col]==cat]]).reset_index(drop=True)
            
            # removendo categorias que não serão oversampled
            df_ = df_[df[oversample_col]!=cat].reset_index()
    
    # Separando coluna de oversampling
    X = df_.drop(columns=[oversample_col])
    y = df_[oversample_col]

    # Aplicando Random Oversampling
    oversampler = RandomOverSampler(sampling_strategy='auto', random_state=42)
    X_resampled, y_resampled = oversampler.fit_resample(X, y)

    df_resampled = pd.DataFrame(X_resampled, columns=X.columns)
    df_resampled[oversample_col] = y_resampled

    # Exibir o DataFrame balanceado
    print(df_resampled[oversample_col].value_counts())
    
    # inserindo de volta as categorias que foram separadas
    if not df_cat.empty:
        df_resampled = pd.concat([df_resampled, df_cat]).reset_index(drop=True)
        
    return df_resampled

In [118]:
def apply_random_undersampling(df, undersample_col='', strategy={}):
    '''
    df: dataframe with all columns
    undersample_col: column to apply the undersampling
    srtategy: dictionary with the category as key and number of samples requested as value
    '''

    X = df.drop(columns=[undersample_col])
    y = df[undersample_col]

    # aplicando undersampling
    undersampler = RandomUnderSampler(sampling_strategy=strategy, random_state=42)
    X_resampled, y_resampled = undersampler.fit_resample(X, y)

    df_resampled = pd.DataFrame(X_resampled, columns=X.columns)
    df_resampled[undersample_col] = y_resampled

    # Exibir o DataFrame balanceado e a distribuição de classes
    print(df_resampled[undersample_col].value_counts())
    
    return df_resampled

In [3]:
# col_raca = 'P1_c'
# col_genero = 'P1_b'
# col_idade ='P1_a_1'
# # col_regiao = 'P1_i_2'

# cols_resampling = [col_raca, col_genero, col_idade ] # , col_regiao]

# df_resampled = train.copy()

# for col in cols_resampling:
#     df_resampled = apply_random_oversampling(df_resampled, oversample_col=col)

## Teste pulando oversampling

In [4]:
df_resampled = train.copy()

In [122]:
# !pip install seaborn

In [5]:
onehot = ["P1_m", "P3_c","P4_a","P4_b", "P4_c", "P4_d", "P4_e", "P4_g", "P4_j", "P5_b", "P6_a", "P6_b", "P6_g", "P6_h", "P7_a", "P7_b", "P7_d", "P8_a", "P8_b", "P8_c", "P8_d", "P2_b", "P2_d", "P2_f", "P2_o", "P2_r", "P6_c", "P6_d", "P6_e", "P6_f"]
ordered = ["P1_l", "P2_e", "P2_g", "P2_i", "P2_j", "P2_h"]

# df_feat_selected = df_resampled[ [x +endswith("(OneHot)") for x in onehot ] + [x +endswith("(OrdEnc)") for x in ordered] ]


In [6]:

onehot_cols = [col for col in df_resampled.columns if any(col.startswith(c) for c in onehot) and col.endswith('(OneHot)')]
ordered_cols = [col for col in df_resampled.columns if any(col.startswith(c) for c in ordered) and col.endswith('(OrdEnc)')]
df_feat_selected = df_resampled[onehot_cols+ordered_cols]

In [7]:
df_feat_selected.head()

,P1_m_Ciências Biológicas/ Farmácia/ Medicina/ Área da Saúde(OneHot),P1_m_Ciências Sociais(OneHot),P1_m_Computação / Engenharia de Software / Sistemas de Informação/ TI(OneHot),P1_m_Economia/ Administração / Contabilidade / Finanças/ Negócios(OneHot),P1_m_Estatística/ Matemática / Matemática Computacional/ Ciências Atuariais(OneHot),P1_m_Marketing / Publicidade / Comunicação / Jornalismo(OneHot),P1_m_Outra opção(OneHot),P1_m_Outras Engenharias(OneHot),P1_m_Química / Física(OneHot),P1_m_nan(OneHot),...,P6_g_Presto(OneHot),P6_g_Snowflake(OneHot),P6_g_Teradata(OneHot),P6_g_nan(OneHot),P1_l(OrdEnc),P2_e(OrdEnc),P2_g(OrdEnc),P2_i(OrdEnc),P2_j(OrdEnc),P2_h(OrdEnc)
0,False,False,True,False,False,False,False,False,False,False,...,False,False,False,True,5,4,-1,6,2,10
1,False,False,True,False,False,False,False,False,False,False,...,False,False,False,True,4,0,-1,6,6,9
2,False,False,True,False,False,False,False,False,False,False,...,False,False,False,True,6,2,-1,4,1,9
3,False,False,False,False,False,False,False,False,False,True,...,False,False,False,False,1,-1,1,2,1,5
4,False,False,False,True,False,False,False,False,False,False,...,False,False,False,True,4,-1,1,2,0,3


In [8]:
test_feat_selected = test[onehot_cols+ordered_cols]

In [9]:
X_train = df_feat_selected.drop(columns=['P2_h(OrdEnc)'])
y_train = df_feat_selected['P2_h(OrdEnc)']

X_test = test_feat_selected.drop(columns=['P2_h(OrdEnc)'])
y_test = test_feat_selected['P2_h(OrdEnc)']

In [13]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import VarianceThreshold
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import joblib

In [12]:
# Logistic Regression
params_log_reg={
    'solver': ['lbfgs', 'saga', 'newton-cg', 'liblinear'],
    'C': [0.01, 0.1, 1.0, 10.0],
    'multi_class': ['multinomial', 'ovr'],
    'max_iter': [100, 200, 500],
    'penalty': ['l2'],  # 'l1' pode ser testado com 'liblinear' ou 'saga'
    'tol': [1e-4, 1e-3, 1e-2]
}

log_reg = LogisticRegression()

# Configurar RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=log_reg,
    param_distributions=params_log_reg,
    n_iter=10,  # Número de combinações a testar
    scoring='accuracy',  # Métrica de avaliação
    cv=5,  # Número de folds da validação cruzada
    random_state=42,
    n_jobs=-1  # Usar todos os núcleos disponíveis
)

random_search.fit(X_train, y_train)

print("Melhores hiperparâmetros encontrados:")
print(random_search.best_params_)

print("\nAcurácia obtida:")
print(random_search.best_score_)

c:\Users\vi_ma\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
5 fits failed out of a total of 50.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\vi_ma\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\vi_ma\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py", line 1519, in fit
    multi_class = _check_multi_class(self.multi_class, solver, len(self.classes_))
  File "c:\Users\vi_ma\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py", line 483, in _check_multi_class
    r

Melhores hiperparâmetros encontrados:
{'tol': 0.0001, 'solver': 'saga', 'penalty': 'l2', 'multi_class': 'multinomial', 'max_iter': 500, 'C': 1.0}

Acurácia obtida:
0.3212317587661665


c:\Users\vi_ma\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


In [16]:
# lr_params = {'tol': 0.0001, 'solver': 'saga', 'penalty': 'l2', 'multi_class': 'multinomial', 'max_iter': 500, 'C': 1.0}
log_reg = LogisticRegression(tol= 0.0001, solver='saga', penalty='l2', multi_class='multinomial', max_iter=500, C=1.0)
log_reg.fit(X_train, y_train)

y_pred = log_reg.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average=None, zero_division=0)
recall = recall_score(y_test, y_pred, average=None, zero_division=0)
f1 = f1_score(y_test, y_pred, average=None, zero_division=0)
print(f'Accuracy: {accuracy}\nPrecision: {precision}\nRecall: {recall}\nF1-Score:{f1}')

Accuracy: 0.33648790746582546
Precision: [0.         0.26530612 0.24489796 0.2972973  0.36945813 0.25
 0.42424242 0.32       0.19047619 0.22857143 0.16666667 0.
 0.2       ]
Recall: [0.         0.33333333 0.19047619 0.16666667 0.50675676 0.14414414
 0.57534247 0.38095238 0.10958904 0.19047619 0.07407407 0.
 0.0625    ]
F1-Score:[0.         0.29545455 0.21428571 0.21359223 0.42735043 0.18285714
 0.48837209 0.34782609 0.13913043 0.20779221 0.1025641  0.
 0.0952381 ]


c:\Users\vi_ma\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


In [132]:
# KNN
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)

KNeighborsClassifier()

In [133]:
# Decision Tree
decision_tree = DecisionTreeClassifier()
decision_tree.fit(X_train, y_train)

DecisionTreeClassifier()

In [134]:
# Fit the QDA model with the balanced dataset
qda = QuadraticDiscriminantAnalysis()
qda.fit(X_train, y_train)

/Users/acburihan/opt/anaconda3/envs/pytorch/lib/python3.10/site-packages/sklearn/discriminant_analysis.py:926: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


QuadraticDiscriminantAnalysis()

In [135]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [142]:
models = {
    'QDA': qda,
    'Logistic Regression': logistic_regression,
    'KNN': knn,
    'Decision Tree': decision_tree
}

results = []
for model_name, model in models.items():
    y_pred = model.predict(X_train)
    accuracy = accuracy_score(y_train, y_pred)
    precision = precision_score(y_train, y_pred, average=None, zero_division=0)
    recall = recall_score(y_train, y_pred, average=None, zero_division=0)
    f1 = f1_score(y_train, y_pred, average=None, zero_division=0)
    results.append([model_name, accuracy, precision, recall, f1])

# Store the results in a DataFrame
results_df = pd.DataFrame(results, columns=['Model', 'Accuracy', 'Precision', 'Recall', 'F1-Score'])

# Print the results
results_df

,Model,Accuracy,Precision,Recall,F1-Score
0,QDA,0.430064,"[1.0, 0.9899862970380521, 0.21459662186208217,...","[1.0, 0.8872095220102022, 0.9646812699975388, ...","[1.0, 0.9357843869874957, 0.35109170305676857,..."
1,Logistic Regression,0.773382,"[0.78, 0.922926447574335, 0.6387709723064483, ...","[0.7090909090909091, 0.8913659550349519, 0.777...","[0.7428571428571428, 0.906871696299856, 0.7014..."
2,KNN,0.976137,"[0.9727272727272728, 0.9950938767808284, 0.815...","[0.9727272727272728, 0.996315888909881, 0.9911...","[0.9727272727272728, 0.9957045079065375, 0.894..."
3,Decision Tree,0.985662,"[1.0, 0.9999055445357514, 0.8219164221390266, ...","[1.0, 1.0, 0.9996308146689639, 0.8588074574091...","[1.0, 0.9999527700373116, 0.9021045033039037, ..."


In [141]:
models = {
    'QDA': qda,
    'Logistic Regression': logistic_regression,
    'KNN': knn,
    'Decision Tree': decision_tree
}

results = []
for model_name, model in models.items():
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average=None, zero_division=0)
    recall = recall_score(y_test, y_pred, average=None, zero_division=0)
    f1 = f1_score(y_test, y_pred, average=None, zero_division=0)
    results.append([model_name, accuracy, precision, recall, f1])

# Store the results in a DataFrame
results_df = pd.DataFrame(results, columns=['Model', 'Accuracy', 'Precision', 'Recall', 'F1-Score'])

# Print the results
results_df

,Model,Accuracy,Precision,Recall,F1-Score
0,QDA,0.106204,"[0.0, 0.2222222222222222, 0.1446280991735537, ...","[0.0, 0.05128205128205128, 0.5555555555555556,...","[0.0, 0.08333333333333333, 0.22950819672131148..."
1,Logistic Regression,0.242902,"[0.0, 0.3333333333333333, 0.22784810126582278,...","[0.0, 0.3333333333333333, 0.2857142857142857, ...","[0.0, 0.3333333333333333, 0.2535211267605634, ..."
2,KNN,0.248160,"[0.09090909090909091, 0.2, 0.20754716981132076...","[0.1111111111111111, 0.23076923076923078, 0.17...","[0.09999999999999999, 0.21428571428571427, 0.1..."
3,Decision Tree,0.237645,"[0.0, 0.2558139534883721, 0.25862068965517243,...","[0.0, 0.28205128205128205, 0.23809523809523808...","[0.0, 0.2682926829268293, 0.24793388429752067,..."


In [143]:
print(X_test.shape)

(951, 92)
